In [1]:
from bruker2nifti._metadata import BrukerMetadata
from pathlib import Path
from aind_data_schema.components.coordinates import Rotation3dTransform, Scale3dTransform, Translation3dTransform

import logging
import numpy as np

from datetime import datetime, timedelta

import pickle

In [2]:
log_file_name = "./log_files/log_" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".log"
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler(log_file_name, 'w', 'utf-8')
fh.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
# add the handlers to logger
logger.addHandler(fh)

In [4]:
import json
import numpy

path = Path("./MRI_files/RawData-2023_07_21/RAW/DL_AI2.kX2")

metadata = BrukerMetadata(path)
metadata.parse_scans()
metadata.parse_subject()

TEST_OUTPUT_SUBJECT = "./test_output_subject"
TEST_OUTPUT_SCAN = "./test_output_scan"
TEST_OUTPUT_METADATA = "./test_output_metadata"

with open(TEST_OUTPUT_SUBJECT, "wb") as f:
    pickle.dump(metadata.subject_data, f)

with open(TEST_OUTPUT_SCAN, "wb") as f:
    for scan in metadata.scan_data.keys():
        if isinstance(metadata.scan_data[scan], numpy.ndarray):
            metadata.scan_data[scan] = metadata.scan_data[scan].tolist()
        for stuff in metadata.scan_data[scan].keys():
            if isinstance(metadata.scan_data[scan][stuff], numpy.ndarray):
                metadata.scan_data[scan][stuff] = metadata.scan_data[scan][stuff].tolist()

            for thing in metadata.scan_data[scan][stuff].keys():
                if isinstance(metadata.scan_data[scan][stuff][thing], numpy.ndarray):
                    metadata.scan_data[scan][stuff][thing] = metadata.scan_data[scan][stuff][thing].tolist()
        
    pickle.dump(metadata.scan_data, f)

with open(TEST_OUTPUT_METADATA, "wb") as f:
    pickle.dump(metadata, f)

with open("./test_output_metadata", "wb") as f:
    pickle.dump(metadata, f)

# print(metadata.scan_data)
subj = metadata.read_subject()
# print(metadata.list_scans())
# print(metadata.list_recons("7"))
recon = metadata.read_recon("7", "1")
# print(recon)
# print(subj)

#best to look at are acqp and metadata files
#reco / visupars can sometimes have useful data

# print(metadata.subject_data)
ACQ_Time = metadata.scan_data["1"]["acqp"]["ACQ_time"]
print(ACQ_Time)
datetime_format = "%H:%M:%S %d %b %Y"
start = datetime.strptime(ACQ_Time, datetime_format)
length_format = "%Hh%Mm%Ss%fms"
lenght_str = metadata.scan_data["1"]["method"]["ScanTimeStr"]
print(lenght_str)
length = datetime.strptime(lenght_str, length_format)
end_time = start + timedelta(hours=length.hour, minutes=length.minute, seconds=length.second, microseconds=length.microsecond)
print(start)
print(length)
print(end_time)
ACQ_calib_date = metadata.scan_data["1"]["acqp"]["ACQ_calib_date"]

print(metadata.scan_data["1"]["method"].keys())

11:07:59 21 Jul 2023
0h0m12s800ms
2023-07-21 11:07:59
1900-01-01 00:00:12.800000
2023-07-21 11:08:11.800000
dict_keys(['TITLE', 'JCAMPDX', 'DATATYPE', 'ORIGIN', 'OWNER', 'Method', 'EchoTime', 'RepetitionTime', 'NAverages', 'NRepetitions', 'ScanTimeStr', 'ExcPulseAngle', 'DeriveGains', 'ExcPulseEnum', 'ExcPulse', 'GradCalConst', 'Nucleus1Enum', 'Nucleus1', 'RefAttMod1', 'RefAttCh1', 'RefAttStat1', 'Nucleus2Enum', 'Nucleus3Enum', 'Nucleus4Enum', 'Nucleus5Enum', 'Nucleus6Enum', 'Nucleus7Enum', 'Nucleus8Enum', 'EncUseMultiRec', 'EncActReceivers', 'EncZfRead', 'EncPpiAccel1', 'EncPpiRefLines1', 'EncPftAccel1', 'EncZfAccel1', 'EncOrder1', 'EncStart1', 'EncMatrix', 'EncSteps1', 'EncCentralStep1', 'EncTotalAccel', 'EncNReceivers', 'EncAvailReceivers', 'EncChanScaling', 'OperationMode', 'EffSWh', 'EchoPosition', 'EchoTimeMode', 'ReadDephaseTime', 'ExSliceRephaseTime', 'ReadSpoilerDuration', 'ReadSpoilerStrength', 'SliceSpoilerDuration', 'SliceSpoilerStrength', 'DigAutSet', 'DigQuad', 'DigFilter

In [ ]:
# load pickle files

with open(TEST_OUTPUT_SUBJECT, "rb") as f:
    subject_data = pickle.load(f)
    for key in subject_data.keys():
        print(key, subject_data[key])


TITLE Parameter List
JCAMPDX 4.24
DATATYPE Parameter Values
ORIGIN Bruker BioSpin MRI GmbH
OWNER dhlee
SUBJECT_version_nr 2.0
SUBJECT_id DL_AI2
SUBJECT_patient_instance_uid 2.16.756.5.5.100.380435303.12267.1689961864.4393
SUBJECT_name_string DL_AI2
SUBJECT_name DL_AI2
SUBJECT_dbirth 
SUBJECT_type Human
SUBJECT_sex female
SUBJECT_sex_human Female
SUBJECT_remarks DL_AI2
SUBJECT_study_name 684811_2023_07_21
SUBJECT_study_nr 2.0
SUBJECT_study_instance_uid 2.16.756.5.5.100.380435303.12267.1689961864.4394
SUBJECT_size 0.001
SUBJECT_weight 5.0
SUBJECT_purpose 684811_2023_07_21
SUBJECT_pos_valid Yes
SUBJECT_location DL_AI2
SUBJECT_entry SUBJ_ENTRY_HeadFirst
SUBJECT_position SUBJ_POS_Supine
SUBJECT_date 10:51:04 21 Jul 2023
SUBJECT_abs_date 1689961864.0
SUBJECT_study_coil_config_file EDIT_unknown_unknown_unknown_unknown_1
END 


In [ ]:
# Checking Method Fields
cur_method = metadata.scan_data['7']['method']

# print(metadata.scan_data['7'].keys())
# print(self.cur_method.keys())
print(cur_method['Method'])
# print(self.cur_method['EchoTime'])
# print(self.cur_method['EffectiveTE'])
# print(self.cur_method['RareFactor'])
# print(self.cur_method['RepetitionTime'])
print(cur_method['SpatResol'])
print(type(cur_method['SpatResol'][0]))

RARE
[0.1, 0.1, 0.1]
<class 'float'>


In [ ]:
print(metadata.scan_data['7']['recons']['1']['visu_pars'])
cur_visu_pars = metadata.scan_data['7']['recons']['1']['visu_pars']
print(*cur_visu_pars['VisuCoreOrientation'])
print(cur_visu_pars['VisuCorePosition'])

scan7 = metadata.read_scan('7')


# print(scan7['recons']['1']['visu_pars'].keys())

{'TITLE': 'Parameter List', 'JCAMPDX': 4.24, 'DATATYPE': 'Parameter Values', 'ORIGIN': 'Bruker BioSpin MRI GmbH', 'OWNER': 'dhlee', 'VisuVersion': 1.0, 'VisuUid': '2.16.756.5.5.100.380435303.14105.1689969391.16', 'VisuCreator': 'ParaVision', 'VisuCreatorVersion': '5.1', 'VisuCreationDate': '11:27:33 21 Jul 2023', 'VisuCoreFrameCount': 1.0, 'VisuCoreDim': 3.0, 'VisuCoreSize': array([140., 200., 130.]), 'VisuCoreDimDesc': 'spatial spatial spatial', 'VisuCoreExtent': array([14., 20., 13.]), 'VisuCoreFrameThickness': 13.0, 'VisuCoreUnits': ['mm', 'mm', 'mm'], 'VisuCoreOrientation': array([[ 1.,  0.,  0.,  0.,  0., -1.,  0.,  1.,  0.]]), 'VisuCorePosition': array([[-6.1, -7. ,  7.9]]), 'VisuCoreDataMin': 49.0, 'VisuCoreDataMax': 32766.0, 'VisuCoreDataOffs': 0.0, 'VisuCoreDataSlope': 403.972115083361, 'VisuCoreFrameType': 'MAGNITUDE_IMAGE', 'VisuCoreWordType': '_16BIT_SGN_INT', 'VisuCoreByteOrder': 'littleEndian', 'VisuCoreDiskSliceOrder': 'disk_reverse_slice_order', 'VisuSubjectName': 'DL_A

In [ ]:
from aind_data_schema.core.mri_session import MriScanSequence, ScanType, SubjectPosition
from decimal import Decimal
from aind_data_schema.models.units import MassUnit, TimeUnit

# method
    ##$Method=RARE
    ##$PVM_RareFactor=8,
    ##$PVM_EchoTime=0.01,
    ##$EffectiveTE=(1)
    # 10.6666667
    ##$PVM_RepetitionTime=500,
    ##$PVM_SpatResol=( 3 )
    # 0.1 0.1 0.1

# visu_pars
    ##$VisuCoreOrientation=( 1, 9 )
    # 1 0 0 0 0 -1 0 1 0
    ##$VisuCorePosition=( 1, 3 )
    # -6.1 -7 7.9

# subject
    ##$SUBJECT_position=SUBJ_POS_Supine

In [ ]:
from pydantic import Field, ValidationInfo, field_validator, model_validator
from aind_data_schema.base import AindCoreModel, AindGenericType, AindModel
from typing import List, Literal, Optional
from aind_data_schema.models.process_names import ProcessName


class MRIScan(AindModel):
    """Description of a 3D scan"""

    scan_index: int = Field(..., title="Scan index")
    scan_type: ScanType = Field(..., title="Scan type")
    primary_scan: bool = Field(
        ..., title="Primary scan", description="Indicates the primary scan used for downstream analysis"
    )
    scan_sequence_type: MriScanSequence = Field(..., title="Scan sequence")
    rare_factor: Optional[int] = Field(None, title="RARE factor")
    echo_time: Decimal = Field(..., title="Echo time (ms)")
    effective_echo_time: Optional[Decimal] = Field(None, title="Effective echo time (ms)")
    echo_time_unit: TimeUnit = Field(TimeUnit.MS, title="Echo time unit")
    repetition_time: Decimal = Field(..., title="Repetition time (ms)")
    repetition_time_unit: TimeUnit = Field(TimeUnit.MS, title="Repetition time unit")
    # fields required to get correct orientation
    vc_orientation: Optional[Rotation3dTransform] = Field(None, title="Scan orientation")
    vc_position: Optional[Translation3dTransform] = Field(None, title="Scan position")
    subject_position: SubjectPosition = Field(..., title="Subject position")
    # other fields
    voxel_sizes: Scale3dTransform = Field(..., title="Voxel sizes", description="Resolution")
    processing_steps: List[
        Literal[
            ProcessName.FIDUCIAL_SEGMENTATION,
            ProcessName.REGISTRATION_TO_TEMPLATE,
            ProcessName.SKULL_STRIPPING,
        ]
    ] = Field([])
    additional_scan_parameters: AindGenericType = Field(..., title="Parameters")
    notes: Optional[str] = Field(None, title="Notes", validate_default=True)

    @field_validator("notes", mode="after")
    def validate_other(cls, value: Optional[str], info: ValidationInfo) -> Optional[str]:
        """Validator for other/notes"""

        if info.data.get("scan_sequence_type") == MriScanSequence.OTHER and not value:
            raise ValueError(
                "Notes cannot be empty if scan_sequence_type is Other."
                " Describe the scan_sequence_type in the notes field."
            )
        return value

    @model_validator(mode="after")
    def validate_primary_scan(self):
        """Validate that primary scan has vc_orientation and vc_position fields"""

        if self.primary_scan:
            if not self.vc_orientation or not self.vc_position:
                raise ValueError("Primary scan must have vc_orientation and vc_position")
            
        return self

AttributeError: type object 'ProcessName' has no attribute 'REGISTRATION_TO_TEMPLATE'

: 

In [ ]:
from bruker2nifti._metadata import BrukerMetadata
from pathlib import Path
from aind_data_schema.models.coordinates import Rotation3dTransform, Scale3dTransform, Translation3dTransform
from aind_data_schema.core.mri_session import MRIScan, MriSession, MriScanSequence, ScanType, SubjectPosition
from decimal import Decimal
from aind_data_schema.models.units import MassUnit, TimeUnit
from aind_data_schema.models.devices import Scanner, ScannerLocation, MagneticStrength
from datetime import datetime

import traceback


class MRILoader:
    def __init__(self, data_path):
        self.metadata = BrukerMetadata(path)
        self.metadata.parse_scans()
        self.metadata.parse_subject()
        self.n_scans = self.metadata.list_scans()


    def load_mri_session(self, experimenter: str, primary_scan_number: str, setup_scan_number: str, scan_location: ScannerLocation, magnet_strength: MagneticStrength) -> MRIScan:

        scans = []
        for scan in self.n_scans:
            scan_type = "3D Scan"
            if scan == setup_scan_number:
                scan_type = "Set Up"
            primary_scan = False
            if scan == primary_scan_number:
                primary_scan = True
            new_scan = self.make_model_from_scan(scan, scan_type, primary_scan)
            logging.info(f'loaded scan {new_scan}')

            print("dumped: ", new_scan.model_dump_json())
            scans.append(new_scan)

        # scanner_dict = {
        #         "name":"test_scanner",
        #         "scanner_location":scan_location,
        #         "magnetic_strength":magnet_strength, 
        #         "magnetic_strength_unit":"T", 
        #         }

        # session_dict = {
        #     "subject_id":"",
        #     "session_start_time":datetime.now(), 
        #     "session_end_time":datetime.now(),
        #     "experimenter_full_name":experimenter, 
        #     "protocol_id":"",
        #     "iacuc_protocol":"",
        #     # animal_weight_prior=,
        #     # animal_weight_post=,
        #     # weight_unit=, 
        #     # anaesthesia=,
        #     "mri_scanner": scanner_dict,
        #     "scans":scans,
        #     "notes":"none"
        # }


        # logging.info(f'loaded session: {session_dict}')
        logging.info(f'loaded scans: {scans}')
        # return MriSession.model_validate(session_dict)
        return MriSession(
            subject_id="",
            session_start_time=datetime.now(), 
            session_end_time=datetime.now(),
            experimenter_full_name=experimenter, 
            protocol_id="",
            iacuc_protocol="",
            # animal_weight_prior=,
            # animal_weight_post=,
            # weight_unit=, 
            # anaesthesia=,
            mri_scanner=Scanner(
                name="test_scanner",
                scanner_location=scan_location,
                magnetic_strength=magnet_strength, 
                magnetic_strength_unit="T", 
            ),
            scans=scans,
            notes="none"
        )
    

    def make_model_from_scan(self, scan_index: str, scan_type, primary_scan: bool) -> MRIScan:
        logging.info(f'loading scan {scan_index}')   

        self.cur_visu_pars = self.metadata.scan_data[scan_index]['recons']['1']['visu_pars']
        self.cur_method = self.metadata.scan_data[scan_index]['method']

        subj_pos = self.metadata.subject_data["SUBJECT_position"]
        if 'supine' in subj_pos.lower():
            subj_pos = 'Supine'
        elif 'prone' in subj_pos.lower():
            subj_pos = 'Prone'

        scan_sequence = MriScanSequence.OTHER
        notes = None
        if 'RARE' in self.cur_method['Method']:
            scan_sequence = MriScanSequence(self.cur_method['Method'])
        else:
            notes = f"Scan sequence {self.cur_method['Method']} not recognized"

        rare_factor = None
        if 'RareFactor' in self.cur_method.keys():
            rare_factor = self.cur_method['RareFactor']

        if 'EffectiveTE' in self.cur_method.keys():
            eff_echo_time = Decimal(self.cur_method['EffectiveTE'])
        else:
            eff_echo_time = None

        rotation=self.cur_visu_pars['VisuCoreOrientation']
        if rotation.shape == (1,9):
            rotation=Rotation3dTransform(rotation=rotation.tolist()[0])
        else:
            rotation = None
        
        translation=self.cur_visu_pars['VisuCorePosition']

        if translation.shape == (1,3):
            translation=Translation3dTransform(translation=translation.tolist()[0])
        else:
            translation = None

        scale=self.cur_method['SpatResol'].tolist()
        while len(scale) < 3:
            scale.append(0)
            # scale = None
        
        scale = Scale3dTransform(scale=scale)

        try:
            return MRIScan(
                scan_index=scan_index,
                scan_type=ScanType(scan_type), # set by scientists
                primary_scan=primary_scan, # set by scientists
                scan_sequence_type=scan_sequence, # method ##$Method=RARE,
                rare_factor=rare_factor, # method ##$PVM_RareFactor=8,
                echo_time=self.cur_method['EchoTime'], # method ##$PVM_EchoTime=0.01,
                effective_echo_time=eff_echo_time, # method ##$EffectiveTE=(1)
                # echo_time_unit=TimeUnit(), # what do we want here?
                repetition_time=self.cur_method['RepetitionTime'], # method ##$PVM_RepetitionTime=500,
                # repetition_time_unit=TimeUnit(), # ditto
                vc_orientation=rotation,# visu_pars  ##$VisuCoreOrientation=( 1, 9 )
                vc_position=translation, # visu_pars ##$VisuCorePosition=( 1, 3 )
                subject_position=SubjectPosition(subj_pos), # subject ##$SUBJECT_position=SUBJ_POS_Supine,
                voxel_sizes=scale, # method ##$PVM_SpatResol=( 3 )
                processing_steps=[],
                additional_scan_parameters={},
                notes=notes, # Where should we pull these?
            )      
        except Exception as e:
            logging.error(traceback.format_exc())
            logging.error(f'Error loading scan {scan_index}: {e}') 

In [ ]:
loader = MRILoader(path)
scan7 = loader.make_model_from_scan('5', '3D Scan', True)
print(scan7)

scan_index=5 scan_type='3D Scan' primary_scan=True scan_sequence_type='RARE' rare_factor=4 echo_time=Decimal('5.33333333333333') effective_echo_time=Decimal('10.6666666666666998253276688046753406524658203125') echo_time_unit=<TimeUnit.MS: 'millisecond'> repetition_time=Decimal('500.0') repetition_time_unit=<TimeUnit.MS: 'millisecond'> vc_orientation=Rotation3dTransform(type='rotation', rotation=[Decimal('1.0'), Decimal('0.0'), Decimal('0.0'), Decimal('0.0'), Decimal('0.0'), Decimal('-1.0'), Decimal('0.0'), Decimal('1.0'), Decimal('0.0')]) vc_position=Translation3dTransform(type='translation', translation=[Decimal('-6.1'), Decimal('-7.1'), Decimal('8.1')]) subject_position='Supine' voxel_sizes=Scale3dTransform(type='scale', scale=[Decimal('0.5'), Decimal('0.4375'), Decimal('0.52')]) processing_steps=[] additional_scan_parameters=AindGeneric() notes=None


In [ ]:
session = loader.load_mri_session(["Mae"], "7", "1", ScannerLocation.FRED_HUTCH, MagneticStrength.MRI_7T)

dumped:  {"scan_index":1,"scan_type":"Set Up","primary_scan":false,"scan_sequence_type":"Other","rare_factor":null,"echo_time":"3.42","effective_echo_time":null,"echo_time_unit":"millisecond","repetition_time":"100.0","repetition_time_unit":"millisecond","vc_orientation":null,"vc_position":null,"subject_position":"Supine","voxel_sizes":{"type":"scale","scale":["0.234375","0.234375","0"]},"processing_steps":[],"additional_scan_parameters":{},"notes":"Scan sequence FLASH not recognized"}
dumped:  {"scan_index":2,"scan_type":"3D Scan","primary_scan":false,"scan_sequence_type":"RARE","rare_factor":8,"echo_time":"5.49333333333333","effective_echo_time":"5.49333333333332962666872845147736370563507080078125","echo_time_unit":"millisecond","repetition_time":"2000.0","repetition_time_unit":"millisecond","vc_orientation":null,"vc_position":null,"subject_position":"Supine","voxel_sizes":{"type":"scale","scale":["0.09765625","0.1953125","0"]},"processing_steps":[],"additional_scan_parameters":{},"

In [ ]:
session.write_standard_file(output_directory=Path("./output"), prefix=Path("test"))